# Verlustprävention an Selbstbedienungskassen im Einzelhandel

## Analyse der Trainingsdaten: Repräsentativität der Kontrollgruppe

Ziel: Wir prüfen, ob sich die gelabelten Transaktionen (kontrolliert) systematisch von den nicht gelabelten unterscheiden.

## 1. Warum ist das wichtig?
Ein verzerrter Trainingsdatensatz könnte dazu führen, dass ein Machine-Learning-Modell falsche Muster lernt – z. B. wer *kontrolliert wurde*, nicht wer *betrügt*.

## 2. Welche Tests wurden durchgeführt?

- **t-Test** für Zahlenwerte (Einkaufssumme, Artikelanzahl, Dauer)
- **Chi²-Test** für Kategorien (Filiale, Kasse, Uhrzeit, Zahlungsmethode)

## 3. Ergebnisse im Überblick


## Test der Datei "transactions_train_3.parquet" auf statistische Signifikanz 

In [1]:
import pandas as pd
from scipy.stats import ttest_ind, chi2_contingency
# Datei einlesen
train = pd.read_parquet("transactions_train_3.parquet")

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481783 entries, 0 to 1481782
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   id                 1481783 non-null  object        
 1   store_id           1481783 non-null  object        
 2   cash_desk          1481783 non-null  int64         
 3   transaction_start  1481783 non-null  datetime64[ns]
 4   transaction_end    1481783 non-null  datetime64[ns]
 5   total_amount       1481783 non-null  float64       
 6   n_lines            1481783 non-null  int64         
 7   payment_medium     1481783 non-null  object        
 8   customer_feedback  104720 non-null   float64       
 9   damage             148025 non-null   float64       
 10  label              1481783 non-null  object        
dtypes: datetime64[ns](2), float64(3), int64(2), object(4)
memory usage: 124.4+ MB


In [3]:
train.label.value_counts()

label
UNKNOWN    1333758
NORMAL      143369
FRAUD         4656
Name: count, dtype: int64

In [4]:
import pandas as pd
from scipy.stats import ttest_ind, chi2_contingency

# -------------------------------------
# Daten laden und Feature Engineering
# -------------------------------------

train['transaction_duration'] = (train['transaction_end'] - train['transaction_start']).dt.total_seconds()
train['hour_of_day'] = train['transaction_start'].dt.hour
train['is_labeled'] = train['label'] != 'UNKNOWN'

# -------------------------------------
# Spalten definieren
# -------------------------------------
numerical_columns = ['total_amount', 'n_lines', 'customer_feedback', 'transaction_duration']
categorical_columns = ['cash_desk', 'payment_medium', 'store_id','hour_of_day']

# -------------------------------------
# Tests durchführen
# -------------------------------------
results = []

# --- Numerische Spalten ---
for col in numerical_columns:
    labeled = train[train['is_labeled']][col].dropna()
    unlabeled = train[~train['is_labeled']][col].dropna()

    if len(labeled) > 0 and len(unlabeled) > 0:
        # Welch's t-Test
        stat, p = ttest_ind(labeled, unlabeled, equal_var=False)
        
        # Mittelwerte und Standardabweichungen
        mean_labeled = labeled.mean()
        mean_unlabeled = unlabeled.mean()
        std_labeled = labeled.std()
        std_unlabeled = unlabeled.std()
        n_labeled = len(labeled)
        n_unlabeled = len(unlabeled)
        
        results.append({
            'Spalte': col,
            'Test': 't-Test',
            'p-Wert': p,
            'Mittelwert (labeled)': mean_labeled,
            'Mittelwert (unlabeled)': mean_unlabeled,
            'Std-Abw (labeled)': std_labeled,
            'Std-Abw (unlabeled)': std_unlabeled,
            
        })

# --- Kategoriale Spalten ---
for col in categorical_columns:
    contingency = pd.crosstab(train['is_labeled'], train[col])

    if contingency.shape[1] > 1:
        chi2, p, _, _ = chi2_contingency(contingency)

        results.append({
            'Spalte': col,
            'Test': 'Chi²-Test',
            'p-Wert': p,
            'Kontingenztabelle': contingency  # Speichere die gesamte Tabelle
        })

# -------------------------------------
# Ergebnisse aufbereiten
# -------------------------------------
# Ergebnisse in DataFrame
results_train = pd.DataFrame(results)

# Ausgabe trennen: Numerisch und Kategorial
numerical_results = results_train[results_train['Test'] == 't-Test']
categorical_results = results_train[results_train['Test'] == 'Chi²-Test']

# Ausgabe für numerische Variablen
print("\nNumerische Spalten (t-Tests):")
display(numerical_results[['Spalte', 'p-Wert', 'Mittelwert (labeled)', 'Mittelwert (unlabeled)', 'Std-Abw (labeled)', 'Std-Abw (unlabeled)']].sort_values(by='p-Wert'))

# Ausgabe für kategoriale Variablen
print("\nKategoriale Spalten (Chi²-Tests):")
for idx, row in categorical_results.iterrows():
    print(f"\nSpalte: {row['Spalte']} (Chi²-Test)")
    print(f"p-Wert: {row['p-Wert']:.5f}")
    print("Kontingenztabelle:")
    print(row['Kontingenztabelle'])



Numerische Spalten (t-Tests):


,Spalte,p-Wert,Mittelwert (labeled),Mittelwert (unlabeled),Std-Abw (labeled),Std-Abw (unlabeled)
3,transaction_duration,0.185389,77.807475,77.541994,73.202614,72.895636
1,n_lines,0.355874,10.603607,10.575406,11.155176,11.101239
2,customer_feedback,0.671868,9.326005,9.318636,1.699571,1.715356
0,total_amount,0.750073,98.509750,98.413698,110.079582,109.943709



Kategoriale Spalten (Chi²-Tests):

Spalte: cash_desk (Chi²-Test)
p-Wert: 0.48348
Kontingenztabelle:
cash_desk        0       1       2       3
is_labeled                                
False       334163  333302  333264  333029
True         37284   37093   36895   36753

Spalte: payment_medium (Chi²-Test)
p-Wert: 0.63991
Kontingenztabelle:
payment_medium    CASH  CREDIT_CARD
is_labeled                         
False           132870      1200888
True             14689       133336

Spalte: store_id (Chi²-Test)
p-Wert: 0.06562
Kontingenztabelle:
store_id    3fffea06-686f-42bd-8362-818af86b48a9  \
is_labeled                                         
False                                     209772   
True                                       23110   

store_id    46e6da32-f4b0-40f3-ada7-fc6ca81ed85d  \
is_labeled                                         
False                                     339525   
True                                       37921   

store_id    581831fc-6a03-4e3

## 4. Fazit

- Die meisten Merkmale zeigen **keine signifikanten Unterschiede** zwischen kontrollierten und nicht kontrollierten Transaktionen.
- Nur bei `store_id` gibt es **einen leichten Hinweis**, dass einzelne Filialen unterschiedlich häufig kontrollieren.

**Die gelabelten Daten sind damit als Trainingsdaten gut geeignet.**
